<a href="https://colab.research.google.com/github/gladson-sza/chatbot_telegram/blob/master/telegram_chatbot_with_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
!pip install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215251 sha256=c7b42a7ab83aab08cce554ec7f9ea5346f653822ea0302d3e5ce6fcd9b6f7d2d
  Stored in directory: /root/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
Successfully built pytelegrambotapi


In [31]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [79]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.5 MB/s eta 0:00:00


In [32]:
!git clone https://github.com/gladson-sza/atrativos_turisticos_dataset.git

fatal: destination path 'atrativos_turisticos_dataset' already exists and is not an empty directory.


In [33]:
!python -m spacy download pt_core_news_sm

2023-11-06 23:38:17.587489: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 23:38:17.587550: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 23:38:17.587575: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-06 23:38:18.483537: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 111.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [34]:
nlp = spacy.load('pt_core_news_sm')

In [35]:
dataset = pd.read_csv('/content/atrativos_turisticos_dataset/dataset.csv', delimiter=';')

In [36]:
dataset.head()

,Pergunta,Resposta
0,Bom dia!,Bom dia! Como posso ajudar você hoje?
1,Boa tarde!,Boa tarde! Estou à disposição para responder à...
2,Boa noite!,Boa noite! Em que posso ser útil?
3,Oi!,Olá! Como posso ser útil a você?
4,"Olá, viajante!",Olá! Estou aqui para fornecer informações sobr...


In [80]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from unidecode import unidecode

nltk.download('punkt')
nltk.download('stopwords')

def preprocessamento(text):
    # Remover caracteres especiais, números e converter para minúsculas
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remover acentos
    text = unidecode(text)

    # Tokenização
    words = word_tokenize(text)

    # Remover stopwords
    stop_words = set(stopwords.words('portuguese'))
    words = [word for word in words if word not in stop_words]

    # Juntar as palavras novamente em uma string
    preprocessed_text = ' '.join(words)

    return preprocessed_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
dataset["Pergunta_Preprocessada"] = dataset["Pergunta"].apply(preprocessamento)

In [87]:
dataset.head()

,Pergunta,Resposta,Pergunta_Preprocessada
0,Bom dia!,Bom dia! Como posso ajudar você hoje?,bom dia
1,Boa tarde!,Boa tarde! Estou à disposição para responder à...,boa tarde
2,Boa noite!,Boa noite! Em que posso ser útil?,boa noite
3,Oi!,Olá! Como posso ser útil a você?,oi
4,"Olá, viajante!",Olá! Estou aqui para fornecer informações sobr...,ola viajante


In [84]:
# Crie um vetorizador TF-IDF para representar o texto
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dataset["Pergunta_Preprocessada"])

In [93]:
def obter_resposta(pergunta):
    pergunta_processada = preprocessamento(pergunta)
    pergunta_vector = vectorizer.transform([pergunta_processada])
    similaridades = cosine_similarity(pergunta_vector, tfidf_matrix)
    pergunta_index = similaridades.argmax()
    return dataset["Resposta"].iloc[pergunta_index]

### Testando Chatbot

In [86]:
flags = ['Fechar', 'Sair', 'Tchau']
end = False

while not end:
    question = input()
    if question in flags:
        end = True
        print('Finalizando Chat')
        continue

    answer = obter_resposta(question)
    print(answer)

Bom dia
Bom dia! Como posso ajudar você hoje?
Como é a amazonia?
A Amazônia é a maior floresta tropical do mundo, localizada na América do Sul. É rica em biodiversidade e abriga uma variedade de espécies de plantas e animais.
Dicas de culinária amazonica
A culinária amazônica é única. Experimente pratos à base de peixes, como o tambaqui e o pirarucu, acompanhados de frutas exóticas, como açaí e cupuaçu.
Fechar
good bye :)


# Integrando com Telegram

In [90]:
import telebot

API_KEY = ''
bot = telebot.TeleBot(API_KEY)

In [94]:
@bot.message_handler(func=lambda message: True)
def default(message):
    resposta = obter_resposta(message.text)

    bot.reply_to(
        message,
        resposta
    )

In [95]:
bot.polling()